In [85]:
import json
import numpy as np
import re
from thai_tokenizer import Tokenizer 
f2 = open('sad_song.json', encoding="utf8")
f = open('songlist.json', encoding="utf8")

Data set preparing
- Extract Data from JSON
    - 5 Happy songs
    - 5 Sad songs

In [86]:
dict_happy = json.load(f)
f.close()
dict_sad = json.load(f2)
f2.close()

In [87]:
print(dict_happy)
print(dict_sad)

{'song1': 'เธอคือท่วงทำนองที่อ่อนหวาน เธอคือสายลมพัดยามอ่อนล้า เธอคือกาแฟในตอนเช้า ยิ่งนานวันยิ่งฉันต้องมีเธอเพิ่มเติมมากกว่า เธอคือไฟที่ให้ไออุ่นฉันในคืนที่หนาวเย็น คือคนเดียวที่อยากจะได้เห็นเมื่อฉันลืมตา ไม่เคยพอกับการได้มีเธอมาใช้ชีวิตด้วยกัน เธอคือดวงตะวัน ที่โลกฉันหมุนรอบเธอ ห้ามใจไม่ให้เจอ ไม่รู้ว่าต้องทำไง เป็นคาเฟอีนที่ฉัน ต้องการเมื่อยามเหนื่อยล้า เธอคือความหวาน ที่ดีต่อหัวใจi', 'song2': "พี่คงจะอยู่ไม่ได้เเล้ว ถ้าไม่มีหนูคืนนี้คงนอนไม่ค่อยจะหลับ อยากคอลหาสักทีได้ไหม ตอนไหนที่หนูมีเวลา ก็คิดถึงนี่นา คิดถึงหนูที่สุด จะหยุดตัวเองยังไงก็ไม่ไหว พี่คงจะอยู่ไม่ได้แล้ว ถ้าไม่มีหนูคืนนี้ยังไงก็นอนไม่หลับ ช่วยบอกรักสักทีได้ไหม ตอนไหนที่หนูมีเวลา อย่าให้ทรมาน คิดถึงหนูทุกวัน Oh Baby you're my special one เพราะหนูสำคัญที่สุดเลย", 'song3': 'ขออยู่ในชีวิตที่เหลือของเธอได้ไหม อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน และเอนไปจุมพิตเธอสักล้านครั้ง อยู่กับฉันไปนาน ๆ นะเธอ', 'song4': 'ตื่นเช้ากว่าวันไหนสดใสกว่าทุกวัน ต้นไม้และลำธารก็ยังดูสดใสไม่สู้เธอ โชว์ยิ้

In [88]:
happy_song = []
sad_song = []
for i in dict_happy:
    happy_song.append(dict_happy[i])
for j in dict_sad:
    sad_song.append(dict_sad[j])

Seperate into Train and Test set

In [89]:
train_set = happy_song[:4] + sad_song[:4]
test_set = happy_song[4:] + sad_song[4:]

In [90]:
train_y = np.append(np.ones((len(happy_song[:4]), 1)), np.zeros((len(sad_song[:4]), 1)), axis=0)
test_y = np.append(np.ones((len(happy_song[4:]), 1)), np.zeros((len(sad_song[4:]), 1)), axis=0)
train_y2 = np.squeeze(train_y)
test_y2 = np.squeeze(test_y)

In [91]:
tokenizer = Tokenizer() #declared Thai Tokenizer as tokenizer

Utility functions.
- Process (get rid of unnecessary  characters or symbols) \n
- build_freq (create frequency dictionary of positvie and negative word appear in dataset)  - [(word,1):frequency] (positive) and [(word,0):frequency] (negative)
- sigmoid (logistic function result in range (0,1))
- gradient descent (reducing cost function)
- extract_features (convert word to vector using frequency dictionary)
- vectorized (convert all of word in dataset to vector [bias(1), positive_count, negative_count])


In [92]:
def Process(sentence):
    i= re.sub(r'[(,)]', '', sentence)
    x = tokenizer(i)
    arr = x.split(' ')
    return arr 

In [93]:

def build_freq(y,x):
    freqs = {}
    for yi,xi in zip(y,x):
        for word in Process(xi):
            pair = (word, yi)
            if pair in freqs:
                freqs[pair] +=1
            else:
                freqs[pair] = 1
    return freqs


In [94]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [110]:

def gradientDescent(x, y, theta, alpha, num_iters):
    m = x.shape[0]

    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)
        J = -1/m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))
        theta += -(alpha/m)*(np.dot(x.transpose(),(h-y)))
        #print("epoch:", i," Cost:", J)
    J = float(J)
    return J, theta

In [111]:
def extract_features(song, freqs, Process=Process):
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = Process(song)
    x = np.zeros(3)  # 3 elements for [bias, positive, negative] counts
    x[0] = 1  # bias term is set to 1

    for word in word_l:
        x[1] += freqs.get((word, 1), 0) 
        x[2] += freqs.get((word, 0), 0)    
    
    x = x[None, :]  # adding batch dimension for further processing
    assert(x.shape == (1, 3))
    return x

In [112]:
def Vectorized(X, freqs_dict,extract_features=extract_features):
    a = []
    for w in X: 
        x = extract_features(w, freqs_dict)
        a.append(x[0])
    a = np.array(a)
    return a

Model Class

In [113]:
class Is_this_a_sad_song:

    def __init__(self,train_X, train_Y, test_X, test_Y) -> None:
        self.X_train = train_X
        self.Y_train = train_Y
        self.X_test = test_X
        self.Y_test = test_Y
        self.tokenizer = Tokenizer()
        self.freqs_dict = build_freq(train_y2,train_X)
        self.theta = np.zeros((3, 1))
        self.J = 0
    
    def train(self):
        x = Vectorized(self.X_train,self.freqs_dict)
        J, theta = gradientDescent(x, train_y, np.zeros((3, 1)), 9e-6, 1500)
        self.J = J
        self.theta = theta

        return self.J,self.theta
    
    def predict(self,word):
        word_vector = extract_features(word, self.freqs_dict)
        y_predict = sigmoid(np.dot(word_vector,self.theta))
        if y_predict >= 0.5:
            return 'is not a sad song :)'
        else:
            return 'is a sad song :('
        
    def accuracy(self,predict=predict):
        y_hat = list()
        for song in self.X_test:
            y_pred = predict(song, self.freqs_dict, self.theta)
            if y_pred > 0.5:
                y_hat.append(1.0)
            else:
                y_hat.append(0.0)
        accuracy = sum(np.squeeze(self.Y_test) == y_hat)/(len(self.X_test))
        print(accuracy)
        return accuracy       
        

Building model

In [114]:
MODEL = Is_this_a_sad_song(train_set,train_y,test_set,test_y)
print("PRE TRAINING 1: LOSS =",MODEL.J,"2: theta =",MODEL.theta.T)
MODEL.train()
print("POST TRAINING 1: LOSS =",MODEL.J,"2: theta =",MODEL.theta.T)
print("accuracy =",MODEL.accuracy)

PRE TRAINING 1: LOSS = 0 2: theta = [[0. 0. 0.]]
POST TRAINING 1: LOSS = 0.013224808143444549 2: theta = [[ 7.84087286e-05  3.80549117e-02 -3.58372945e-02]]
accuracy = <bound method Is_this_a_sad_song.accuracy of <__main__.Is_this_a_sad_song object at 0x0000020A9F65BD90>>


C:\Users\peaks\AppData\Local\Temp\ipykernel_14424\3211945146.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


PREDICTION

In [115]:
pred1 = 'ก็ฉันเมื่อรักใครก็ต้องเป็นคนผิดหวัง'
pred2 = 'เธอคือหวานเย็น ดับร้อนข้างในหัวใจที่ฉันมี'


In [116]:
print("Lyric :",pred1,MODEL.predict(pred1))
print("Lyric :",pred2,MODEL.predict(pred2))



Lyric : ก็ฉันเมื่อรักใครก็ต้องเป็นคนผิดหวัง is a sad song :(
Lyric : เธอคือหวานเย็น ดับร้อนข้างในหัวใจที่ฉันมี is not a sad song :)
